# Strategy Report Notebook
This notebok runs analysis on the Strategy and Allocation Portfolio

## Import Libraries

In [1]:
import pandas as pd
import ipywidgets as widgets
import datapane as dp 
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
from EquityHedging.reporting.excel import sheets
import os

## Compute Equity Hedge Returns for International Benchmark
To compute: Select an Equity Benchmark (**equity_bmk**):  SPTR, M1WD, SX5T



In [2]:
equity_bmk = 'M1WD'
bmks = dm.get_equity_hedge_returns(equity_bmk, only_equity=True)


## Add a Strategy
Add File Name to "io" and the name of the sheet in "sheet_name" in **data=pd.read_excel(io= , sheet_name= )

In [3]:
CWD = os.getcwd()
NEWSTRAT_DATA_FP = '\\EquityHedging\\data\\'
NEW_STRAT_DATA = CWD + NEWSTRAT_DATA_FP + 'new_strats'
data=pd.read_excel(io="Credit Suisse_SX5E_Tail_FSFVA_20210528_UPS Pension.xlsx", sheet_name="data",index_col=0)

## Get Return Data of New Strategy and Merge with Benchmark

In [4]:
data_dict=dm.get_data_dict(data,'index')
ret_strats=ret_strats=dm.merge_dicts(bmks, data_dict)

## View Returns Data

In [5]:
freq_list = ['Daily', 'Weekly', 'Monthly','Quarterly', 'Yearly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(ret_strats[frequency])

interactive(children=(Dropdown(description='frequency', options=('Daily', 'Weekly', 'Monthly', 'Quarterly', 'Y…

todo: show correlation chart, return strats, and hedge metrics

# Compute Correlations

In [14]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict = summary.get_corr_data(ret_strats)

# Display Correlations

In [21]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['corr', 'corr_down', 'corr_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

# Create Report

In [20]:
strat_report = 'Strat_Report'
selloffs = True
rp.generate_strat_report(strat_report, ret_strats, True)

Computing Monthly Analytics...
Computing Weekly Analytics...
Computing Historical SellOffs...
